#Fractals

From Wikipedia, https://en.wikipedia.org/wiki/Fractal: 

    A fractal is a natural phenomenon or a mathematical set that exhibits a repeating pattern that displays at every scale. 
    
We will be generating fractals using the Newton-Raphson root finding algorithm. For more information on this algorithm, see https://en.wikipedia.org/wiki/Newton%27s_method.

Lets import all the needed libraries:

In [1]:
import numpy as np
import pylab as plb
import matplotlib.pyplot as plt
from numpy import pi
from colorsys import hls_to_rgb
from textwrap import wrap

The numpy library holds the matrices we will be using to generate the fractal. It also has the complex number type, which we will need.

The pylab library has the tools to actually take the matrix from numpy and create an image to display on screen.

The pyplot module of matplotlib has the tools to write save the fractal image into a .pdf file.

From numpy, we also explicitly want to use the constant Pi.

The hls_to_rgb function from the colorsys library will auto-magically generate colors for us.

The textwrap library will let us write out the polynomial we are using to generate the fractal in a nice format.

We will be working with polynomials for this exercise. To efficiently store the polynomial function, we will only record the coefficients in descending powers of x. That is, if we have the polynomial

    3x^2 + 2x + 7

we would store this as the array

    [3, 2, 7].
    
If we had the polynomial 

    5x^5 - 3x^3 + x
    
we would store this as the array

    [5, 0, -3, 0, 1, 0]
    
The order is very important here.

Now lets start writing the functions we will need. First, lets write a function that will represent our polynomial. First, the function will take as input the point at which the function will be evaluated (the z in F(z)). Second, we will need to give it the array of coefficients.

In [2]:
# The polynomial
def f(z, coeff):
    '''
        Evaluates a polynomial represented by coeff at the point z.
        Input parameters: 
                z     - the point at which the polynomial will be evaluated
                coeff - the array holding the coefficients of the polynomial in descending powers of x
        Output parameters:
                value - the value of the polynomial at point z
    '''
    value = 0.0
    for i in range(len(coeff)):
        value += coeff[i]*z**(len(coeff)-i)
    return value

Next is a function to represent the derivative of the polynomial.

In [3]:
# The derivative f'(z)
def f_prime(z, coeff):
    '''
        Evaluates the derivative of a polynomial represented by coeff at the point z.
        Input parameters: 
                z     - the point at which the polynomial will be evaluated
                coeff - the array holding the coefficients of the polynomial in descending powers of x
        Output parameters:
                value - the value of the derivative of the polynomial at point z
    '''
    value = 0.0
    for i in range(len(coeff)-1):
        value += coeff[i]*(len(coeff)-i)*z**(len(coeff)-i-1)
    return value

The second derivative will also be needed periodically, so we will need a function for that. Its written here.

In [4]:
# The second derivative f''(z)
def f_doubleprime(z, coeff):
    '''
        Evaluates the second derivative of a polynomial represented by coeff at the point z.
        Input parameters: 
                z     - the point at which the polynomial will be evaluated
                coeff - the array holding the coefficients of the polynomial in descending powers of x
        Output parameters:
                value - the value of the second derivative of the polynomial at point z
    '''
    value = 0.0
    for i in range(len(coeff)-2):
        value += coeff[i]*(len(coeff)-i)*(len(coeff)-i-1)*z**(len(coeff)-i-2)
    return value

We will also need a function that implements the Newton-Raphson algorithm. It is written below.

In [5]:
# Newton-Raphson Solver
def newton(x,coeff):
    '''
        Finds a root of the polynomial represented by the array coeff.
        Input parameters: 
                x     - the starting guess for the solver
                coeff - the array holding the coefficients of the polynomial in descending powers of x
        Output parameters:
                x     - the calculated root value
    '''
    # Keep running until we are within 1e-12 of the real root
    while abs(f(x,coeff)) >= 1e-12: #Tolerance value hardcoded
        
        # Update procedure if derivative is zero (this avoids dividing by zero)
        if f_prime(x,coeff) == 0:
            x = x - f(x)/(f_prime(x)**2-f(x)*f_doubleprime(x))

        # Standard update procedure    
        else:
            x = x - f(x,coeff)/f_prime(x,coeff)           

    # Return the root we found
    return x

The final piece of this puzzle is a function to provide all the colors we will need. It is given here.

In [6]:
# Coloring complex valued things
def colorize(z):
    '''
        Takes a complex number and turns it into a rgb color value
        Input parameters: 
                z - the complex number to be colorized
        Output parameters:
                c - the rgb color value of the complex number z
    '''
    r = np.abs(z)
    arg = np.angle(z) 

    h = (arg + pi)  / (2 * pi) + 0.5
    l = 1.0 - 1.0/(1.0 + r**0.3)
    s = 0.8

    c = np.vectorize(hls_to_rgb) (h,l,s) # --> tuple
    c = np.array(c)  # -->  array of (3,n,m) shape, but need (n,m,3)
    c = c.swapaxes(0,2) 
    return c

That is all the functions needed. Now lets prompt the user for a polynomial:

In [7]:
print 'Enter the coefficients of your polynomial (up to degree 10) seperated by spaces. \nFor example the polynomial \n\n\tx^2 + 2x + 3\n\nwould be entered as \"1 2 3\":'
coeff = raw_input().split()

Enter the coefficients of your polynomial (up to degree 10) seperated by spaces. 
For example the polynomial 

	x^2 + 2x + 3

would be entered as "1 2 3":
9 4 7 2 1 6


A few more constants are needed:

In [8]:
left_bc = -1000 # The left boundary for our domain
right_bc = 1000 # The right boundary for our domain
size = 300      # Number of pixels per side of image. !!!! <<<<CAUTION>>>> !!!! Making this larger will 
                # dramatically increase its runtime. You have been warned.

Now to use the input and variables to set up the algorithm:

In [9]:
# Cast the entries as floating point numbers
coeff = [float(i) for i in coeff]

# 2D array to hold data (initialized to zero)
data = np.zeros((size,size), dtype = complex)

# x and y values for image (each represents some number of pixels?)
x = np.linspace(left_bc, right_bc, size)

Everything is ready now. Time to use the Newton-Raphson algorithm (the slow part).

In [10]:
# Calculating roots
for i in range(len(x)):
    for j in range(len(x)):
        data[i][j] = newton(complex(x[i]/float(size),x[j]/float(size)), coeff)

Here we create a nicely formatted string to represent our polynomial.

In [11]:
# Generating a string of the entered polynomial
polynomial = ""
for i in range(len(coeff)):
    if i == 0:
        polynomial += "%d * x^%d " % (coeff[i], len(coeff) - i)
    else:
        polynomial += "+ %d * x^%d " % (coeff[i], len(coeff) - i)

Time to visualize our work!

In [12]:
#Plotting
img = colorize(data)
im = plb.imshow(img)
im.axes.get_xaxis().set_visible(False)
im.axes.get_yaxis().set_visible(False)
plb.title("\n".join(wrap(polynomial)))
plb.savefig('fractal.pdf', bbox_inches='tight')
plt.show()